In [1]:
import json
from BuildGraphModule import *
from unidecode import unidecode
from graviz import graph_processing

In [2]:
with open('paper.json', 'r') as f:
    data = json.load(f)

In [3]:
# Uniform strings - Authors
for i in list(data.keys()):
    try:
        for j in range(len(data[i]['Authors'])):
            t = unidecode(data[i]['Authors'][j])
            t.encode("ascii")
            data[i]['Authors'][j] = t.lower().replace('.','')
            
    except:
        continue

In [4]:
# Uniform strings - EPFL Authors
for i in list(data.keys()):
    dic = []
    try:
        for j in range(len(data[i]['Epfl authors'])):
            string = list(data[i]['Epfl authors'][j].keys())[0]
            t = unidecode(string)
            t.encode("ascii")
            dic += [{t.lower().replace('.','') : list(data[i]['Epfl authors'][j].values())[0]}]
        data[i]['Epfl authors'] = dic
    except:
        continue

In [5]:
authors = get_authors(data, 'Authors')

In [4]:
# All
dictionary_a_id, dictionary_id_a = create_authors_id(authors, 'all')

In [5]:
# All
dict_authors = list_coauthors(data)

In [ ]:
# All
dict_numb_coll = number_collaborations(dict_authors)

In [ ]:
# All
dict_authors_set = set_coauthors(dict_authors)

In [ ]:
# All
edges = create_edges(dict_authors_set, dictionary_a_id)

## EPFL graph: Multiplegraph

In [5]:
authors_epfl = get_authors(data, 'Epfl authors')
# Due to the format which the authors have been saved

# We initialize a new list
epfl_authors = []

# Then for each element in the previous list
for i in authors_epfl:
    # We unpack the content
    epfl_authors += list(i.keys())

In [6]:
# Id-Author / Author-ID
dictionary_a_id_epfl, dictionary_id_a_epfl = create_authors_id(epfl_authors, 'epfl')

In [7]:
# EPFL - COAUTORI NELLA LISTA DI EPFL AUTHORS
dict_authors_epfl = list_coauthors_epfl(data)

In [8]:
# Number of collaborations
dict_numb_coll_epfl = number_collaborations(dict_authors_epfl)

In [9]:
# Coauthors set
dict_authors_set_epfl = set_coauthors(dict_authors_epfl)

In [10]:
# Set of edges
edges_epfl = create_edges(dict_authors_set_epfl, dictionary_a_id_epfl)

In [11]:
G = create_simple_graph(dictionary_id_a_epfl, edges_epfl)

Name: 
Type: Graph
Number of nodes: 13451
Number of edges: 45417
Average degree:   6.7530


In [ ]:
graph_processing.save_graph(G, 'graviz/graph.json')

In [15]:
# Initialize the dictionary
dict_authors_epfl = defaultdict(list)

# For each pubblication
for p in list(data.keys()):
    # If present
    try:
        # Extracte the list of epfl authors
        list_authors_raw = data[p]['Epfl authors']
        # Unpack the list
        list_authors_list = [list(i.keys()) for i in list_authors_raw]

        # Whether the list of authors contains more than one element
        if len(list_authors_list) > 1:
            # Get the final list unpacking elements
            list_authors = [i[0] for i in list_authors_list]
        else:
            # Just get the element as the list
            list_authors = list_authors_list[0]

        # For each author create an instance in the dictionary
        for a in list_authors:
            dict_authors_epfl[a] += [{j:p} for j in list_authors if j != a]

    # If there re not epfl authors continue the loop
    except:
        continue

In [55]:
dict_authors_epfl['catasta michele']

[{'sae-tang abson': '182133'},
 {'mcdowell lucas kelsey': '182133'},
 {'aberer karl': '182133'},
 {'tran le hung': '182131'},
 {'mcdowell lucas kelsey': '182131'},
 {'aberer karl': '182131'},
 {'bonvin nicolas': '146811'},
 {'aberer karl': '205653'},
 {'cudre-mauroux philippe': '205653'},
 {'aberer karl': '202024'},
 {'christodoulou georgios': '202024'},
 {'gavrilovic ivan': '202024'},
 {'hrisafov filip': '202024'},
 {'monney mathieu': '202024'},
 {'ouaazki abdessalam': '202024'},
 {'perovic boris': '202024'},
 {'ranvier jean-eudes marie': '202024'},
 {'vasirani matteo': '202024'},
 {'yan zhixian': '202024'},
 {'jeung ho young': '169190'},
 {'miklos zoltan': '169190'},
 {'aberer karl': '169190'},
 {'yerva surender reddy': '188836'},
 {'aberer karl': '188836'},
 {'vasirani matteo': '208793'},
 {'aberer karl': '208793'},
 {'aberer karl': '196181'},
 {'radu horia': '196181'},
 {'ranvier jean-eudes marie': '196181'},
 {'vasirani matteo': '196181'},
 {'yan zhixian': '196181'},
 {'aberer kar

In [17]:
dict_coautorship_papers = {}
for author in list(dict_authors_epfl.keys()):
    
    dict_papers = defaultdict(list)
    for i in range(len(dict_authors_epfl[author])):

        coauthor = list(dict_authors_epfl[author][i].keys())[0]

        dict_papers[coauthor] += [(dict_authors_epfl[author][i])[coauthor]]

    dict_coautorship_papers[author] = dict_papers

In [88]:
dictionary = {}
dictionary_occ = {}
for i in list(dict_coautorship_papers.keys()):
    dictionary[i] = {}
    dictionary_occ[i] = {}
    for j in list(dict_coautorship_papers[i].keys()):
        dates = defaultdict(list)
        for l in range(len(dict_coautorship_papers[i][j])):
            try:
                dates[data[sorted(dict_coautorship_papers[i][j])[l]]['Publication date']] += [sorted(dict_coautorship_papers[i][j])[l]]
            except:
                continue
        if len(dates) != 0:
            dictionary[i][j] = dates
            dictionary_occurences = {}
            for k,m in dictionary[i][j].items():
                dictionary_occurences[k] = len(m)
            dictionary_occ[i][j] = dictionary_occurences

In [ ]:
# Define an epty set of edges
edges = set()

# For each author
for key in list(dict_authors_set.keys()):       
    # If the list of co-authors is not empty
    if len(dict_authors_set[key]) != 0:
        # Create sorted tuples between the author and his coauthors
        new_edges = [tuple(sorted((dictionary_a_id[key], dictionary_a_id[co]))) for co in dict_authors_set[key]]

        # Update the set in order to not have the double arches
        edges.update(new_edges)

In [90]:
dictionary['catasta michele']

{'aberer karl': defaultdict(list,
             {'2012': ['182131', '182133'],
              '2013': ['188836', '196256'],
              '2014': ['202024'],
              '2015': ['208793']}),
 'christodoulou georgios': defaultdict(list, {'2014': ['202024']}),
 'cudre-mauroux philippe': defaultdict(list, {'2013': ['196256']}),
 'gavrilovic ivan': defaultdict(list, {'2014': ['202024']}),
 'hrisafov filip': defaultdict(list, {'2014': ['202024']}),
 'mcdowell lucas kelsey': defaultdict(list, {'2012': ['182131', '182133']}),
 'monney mathieu': defaultdict(list, {'2014': ['202024']}),
 'ouaazki abdessalam': defaultdict(list, {'2014': ['202024']}),
 'perovic boris': defaultdict(list, {'2014': ['202024']}),
 'ranvier jean-eudes marie': defaultdict(list, {'2014': ['202024']}),
 'sae-tang abson': defaultdict(list, {'2012': ['182133']}),
 'tran le hung': defaultdict(list, {'2012': ['182131']}),
 'vasirani matteo': defaultdict(list,
             {'2014': ['202024'], '2015': ['208793']}),
 'yan zhi

In [108]:
dictionary_occ['catasta michele']

{'aberer karl': {'2012': 2, '2013': 2, '2014': 1, '2015': 1},
 'christodoulou georgios': {'2014': 1},
 'cudre-mauroux philippe': {'2013': 1},
 'gavrilovic ivan': {'2014': 1},
 'hrisafov filip': {'2014': 1},
 'mcdowell lucas kelsey': {'2012': 2},
 'monney mathieu': {'2014': 1},
 'ouaazki abdessalam': {'2014': 1},
 'perovic boris': {'2014': 1},
 'ranvier jean-eudes marie': {'2014': 1},
 'sae-tang abson': {'2012': 1},
 'tran le hung': {'2012': 1},
 'vasirani matteo': {'2014': 1, '2015': 1},
 'yan zhixian': {'2014': 1},
 'yerva surender reddy': {'2013': 1}}

In [233]:
G_epfl = nx.MultiGraph()

In [234]:
G_epfl.add_nodes_from(list(dictionary_id_a_epfl.keys()))

In [235]:
G_epfl.add_edges_from(list(edges_epfl))

In [236]:
new_edges = []
to_remove = []
for i,j in list(edges_epfl):
    
    try:    
        for k in list(dictionary_occ[dictionary_id_a_epfl[i]][dictionary_id_a_epfl[j]].keys()):
        #print (list(dictionary_occ[dictionary_id_a_epfl[i]][dictionary_id_a_epfl[j]].keys()))
        #print (i,j)
            new_edges += [(i, j, dict(year = k))]
            to_remove += [(i,j)]
    except:
        continue

In [237]:
G_epfl.add_edges_from(new_edges)
#G_epfl.remove_edges_from(set(to_remove))

In [238]:
dictionary_occ[dictionary_id_a_epfl[7884]]['agir berker']

{'2012': 1, '2014': 1, '2016': 1}

In [239]:
dictionary_a_id_epfl['agir berker']

9818

In [240]:
for i,j in G_epfl.edges():
    for k in G_epfl[i][j]:
        #print (k)
        if k == 0:
            G_epfl[i][j][k]['weight'] = dict_numb_coll_epfl[dictionary_id_a_epfl[i]][dictionary_id_a_epfl[j]]
        else:
            #print (G_epfl[i][j][k]['year'])
            G_epfl[i][j][k]['weight'] = dictionary_occ[dictionary_id_a_epfl[i]][dictionary_id_a_epfl[j]][G_epfl[i][j][k]['year']]
    

In [241]:
G_epfl[7884]#[1]#['weight'] = 5

{195: {0: {'weight': 1}, 1: {'weight': 1, 'year': '2007'}},
 213: {0: {'weight': 12},
  1: {'weight': 2, 'year': '2012'},
  2: {'weight': 2, 'year': '2013'},
  3: {'weight': 1, 'year': '2014'},
  4: {'weight': 1, 'year': '2015'}},
 660: {0: {'weight': 1}, 1: {'weight': 1, 'year': '2013'}},
 710: {0: {'weight': 53},
  1: {'weight': 1, 'year': '2008'},
  2: {'weight': 11, 'year': '2003'},
  3: {'weight': 6, 'year': '2002'},
  4: {'weight': 1, 'year': '2001'},
  5: {'weight': 7, 'year': '2006'},
  6: {'weight': 5, 'year': '2004'},
  7: {'weight': 12, 'year': '2005'},
  8: {'weight': 1, 'year': '2007'}},
 739: {0: {'weight': 1}, 1: {'weight': 1, 'year': '2013'}},
 974: {0: {'weight': 1}},
 1182: {0: {'weight': 1}, 1: {'weight': 1, 'year': '2014'}},
 1227: {0: {'weight': 4}, 1: {'weight': 2, 'year': '2007'}},
 1264: {0: {'weight': 13},
  1: {'weight': 2, 'year': '2005'},
  2: {'weight': 5, 'year': '2003'},
  3: {'weight': 5, 'year': '2004'}},
 1286: {0: {'weight': 1}, 1: {'weight': 1, 'year